# Project NO.1 Reed Solomon Code

In [19]:
import random
import numpy as np
import math
import copy
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
random.seed(datetime.now())

In [23]:
def array2int(array):
    tmp = 0
    for idx in range(array.size):
        tmp = tmp + 2**idx*array[idx]
    return tmp

## Systemic Encoding:
### g(x) is quite different :
- Build up $GF(2^8)$ Table:
    - GF_a_exp()   ............ look up table
    - GF_log_a()   ............ inverison

- Generate $g(x)=(x+\alpha)(x+\alpha^2)(x+\alpha^3)(x+\alpha^4)...(x+\alpha^{2t})$

## Build Up $GF(2^8)$ Table

### 用 array element 方式去 建構 $GF(2^8)$
- Pro: 直觀，可以跟課本對照
- Con: 效率低落

In [1]:
def GF_a_exp(alpha,exp,GF_px):# Verified
    # alpha: generator
    # exp  : n of GF(2^n)
    # GF_px: primitive polynomial
    
    GF_2_8 = {}
    tmp = copy.deepcopy(alpha)
    
    GF_2_8[0] = np.asarray([1]+[0]*(alpha.size-1)) # 1st element of any field would be [1,0,0,...]
    # iterate from alpha to alpha^{2^exp-1}, since alpha^{2^exp -1} is 1 , the same as [1,0,0,...] 
    # so we only iterate from alpha to alpha^{2^exp-2}
    
    for i in range(2**exp-2): 
        # add into GF_Table
        GF_2_8[i+1]=copy.deepcopy(tmp)
        # Cirular Shift and check whether need to modulo primitive polynomial
        if(tmp[-1] == 1):
            # Need Modulo Primitive Polynomial
            # Shift
            tmp[1:8] = tmp[0:7]
            tmp[0] = 0
            # Modulo
            tmp = (tmp + GF_px[0:8])%2# Bit-wise add in np.array => Same as the xor
        else:
            # Shift Only
            tmp[1:8] = tmp[0:7]
            tmp[0] = 0
    return GF_2_8

## 用 binary 方式去建構 GF(2^8)
- Pro : memory efficient, high performance
- Con : not intuitive to comprehend

In [2]:
# Turn into binary form of primitive polynomial and alpha
def GF_table_binary_approach(alpha,irreducible,n):# Verified
    table = dict()
    # 1st element is identity element
    table[0] = 1
    tmp = alpha
    # iterate from 1 to 2**n-2, since the 2**n-1 element is 1
    for i in range(1,2**n-1):
        if ( tmp & (2**n) ) :# if current value need modulo
            # Modulo => Add to table => Shift
            tmp = tmp ^ irreducible
            table[i] = tmp
            tmp = tmp << 1
        else:
            table[i] = tmp
            tmp = tmp << 1
    return table

## Build up a Look Up Table 
- GF_a_exp_Table_int
    - input : exponenet
    - output: integer representative of polynomial
- GF_log_a_Table_int
    - input : integer representative of polynomial
    - output: exponenet 

In [3]:
def GF_log_a_table_gen(GF_a_exp_Table_int):
    log_Table = {int(value):int(key) for key,value in GF_a_exp_Table_int.items()}
    # Since log_Table could not find log(0) , we define log(0) would be none in exponenet form
    log_Table[0]=None
    return log_Table

# Basic Galois Field Operation

- GF_mult()
- Poly_Mult_GF()
- GF_inv()
- Poly_divide_GF()
    - Poly_deg()  
        - return the degree of polynomial
    - Poly_decorater() 
        - return the simplify polynomial
- Poly_Add_GF()

### GF_mult()

### result = $\alpha^{x\_exp}\alpha^{y\_exp}=\alpha^{(x\_exp+y\_exp)\%(2^n-1)}$

In [36]:
def GF_mult(x,y,n,GF_log_a_Table_int):# Verified
    n = int(n)
    #  x,y in GF(2^n)
    
    if x == 0 or y == 0 :# for 0 case
        return 0
        
    modulus = (1<<n)-1
    x_exp = GF_log_a_Table_int[x]
    y_exp = GF_log_a_Table_int[y]
    
    result_exp = (x_exp + y_exp)%(modulus) # Because x^(2**n-1) = 1, so (2**n -1) equivalent to 0
    return GF_a_exp_Table_int[result_exp]

### Poly_Mult_GF()

In [5]:
def Poly_Mult_GF(poly1,poly2,n):
    poly1_size = len(poly1)
    poly1_deg  = poly1_size-1
    poly2_size = len(poly2)
    poly2_deg  = poly2_size-1
    
    result = [0]*(poly1_size+poly2_size-1)
    
    for idx1 in range(poly1_size):
        for idx2 in range(poly2_size):
            current_idx = idx1 + idx2
            result[current_idx]= result[current_idx] ^ GF_mult(poly1[idx1],poly2[idx2],n)
    return result    

### GF_inv()
- Look Up Table Version
- Extended Euclidean Version

In [6]:
# Extended Euclidean Version
#def GF_inv(x,n):# Error Overhere
#    #a = 2**n
#    #b = x
#    #r_prev = a
#    #r_cur  = b
#    #s_prev = 1
#    #s_cur  = 0
#    #t_prev = 0
#    #t_cur  = 1
#    #itr = 0
#    #while r_cur != 0:
#    #    itr = itr +1
#    #    q = r_prev//r_cur
#    #    r_cur,r_prev = r_prev - q*r_cur, r_cur
#    #    s_cur,s_prev = s_prev - q*s_cur, s_cur
#    #    t_cur,t_prev = t_prev - q*t_cur, t_cur
#    ## x inverse = t_prev
#    ## remainder = r_prev
#    #return s_prev, t_prev, r_prev
#    exp = GF_log_a_Table_int[x]
#    return GF_a_exp_Table_int[(-exp)%(2**n-1)]

In [7]:
# Look Up Table Version
def GF_inv(x,n):# Error Overhere
    if x == 0:
        return 0
    else:
        exp = GF_log_a_Table_int[x]
        return GF_a_exp_Table_int[(-exp)%(2**n-1)]

### Poly_Divide_GF()

In [8]:
def poly_deg(poly):
    # return the degree of polynomial
    size = len(poly)
    if size == 0:
        return None
    
    for idx in range(size-1,-1,-1):
        if poly[idx] == 0:
            continue
        else:
            return idx
    return 0

In [9]:
def Poly_decorator(poly):
    # truncate the redundant zero in higher degree
    size = len(poly)
    for i in range(size-1,-1,-1):
        if poly[i] == 0:
            continue
        else:
            return poly[:i+1]
    return [0]

In [10]:
def Poly_Divide_GF(dividend,divisor,n):
    dividend = Poly_decorator(dividend)
    divisor  = Poly_decorator(divisor)
    dividend_size = len(dividend)
    divisor_size  = len(divisor)
    # Change into decreasing order {Highest Degree to Lowest Degree} for long division
    remainder = np.asarray(dividend[::-1])
    divisor   = np.asarray(divisor[::-1])
    
    # deg(quotient) = (deg(dividend)-deg(divisor))
    # So the size of the quiteint would be deg(quotient)+1
    # deg(quotient)+1 iteration
    quotient = []
    for idx in range(dividend_size-divisor_size+1):

        if remainder[idx] == 0:
            quotient.append(0)
            continue
        else:
            # q = 1/divisor[0] x remainder's highest degree coefficient, GF's inversion and Multiplication
            q = GF_mult(GF_inv(divisor[0],n), remainder[idx],n)  
            quotient.append(q)
            # q * divisor in GF's muliplication
            q_divisor = [ GF_mult(q,val,n=n) for val in divisor]
            # Since we are in GF(2^n), so substraction is the same as addition
            remainder[idx:idx+divisor_size] = Poly_Add_GF(remainder[idx:idx+divisor_size], q_divisor)
    
    # quotient type: list       
    # remainder type : from numpy arrray to list
    return quotient[::-1] ,remainder[::-1][:divisor_size-1].tolist()

### Poly_Add_GF()

In [11]:
def Poly_Add_GF(poly1,poly2):#Verified
    if len(poly1) > len(poly2):
        large_poly = poly1
        small_poly = poly2
    else:
        large_poly = poly2
        small_poly = poly1
        
    result = large_poly
    for idx,val in enumerate(small_poly):
        result[idx] = result[idx] ^ small_poly[idx]
    return result

### Some Debug tool to view the exponenet of all the coefficient

In [12]:
def Poly_exp_view(poly):
    print( [GF_log_a_Table_int[val] for val in poly] )
    return 

## Generate $g(x)=(x+\alpha)(x+\alpha^2)(x+\alpha^3)(x+\alpha^4)...(x+\alpha^{2t})$

In [13]:
def RS_Generator(msg_len,capability,start_exp,n):# Verified
    # Algorithm: keep multiplying the (a^i+x) where i from 1 to 2t
    generator = [1]
    for i in range(2*capability):
        # since start from alpha^1 to alpha^2t => i+start_exp
        generator = Poly_Mult_GF(generator, [GF_a_exp_Table_int[i+start_exp],1] , n)
    return generator

## Syndrome Polynomial

- received codeword: 

r(x) = $r_0+r_1x+r_2x^2+r_3x^3 + ... r_{n-1}x^{n-1}$

- syndrome:

$S_j=r(\alpha^j)=\sum_{k=0}^{n-1}r_k\alpha^{jk}$, where j from 1 to 2t, $r_k,\alpha\in GF(2^8)$

### Syndrome Polynomial 
- $S(x) = S_1+S_2x+S_3x^2+...S_{2t}x^{2t-1}$

In [14]:
def syndrome_polynomial(received,two_t,n):
    # Turn the received polynomial's coefficient into exponenet form
    exp_form = [GF_log_a_Table_int[val] for val in received]
    S = []
    # Syndrome polynomial's size = 2t
    # index from 1 to 2t
    for i in range(1,two_t+1):
        tmp = 0
        # Computing the Syndrome from recieved polynomial
        for idx,val in enumerate(exp_form):
            if val == None:
                continue
            exp_val = ((idx*i) +val)%(2**n-1) # idx + i means alpha ^ (idx*i)
            tmp = tmp ^ GF_a_exp_Table_int[exp_val]
        S.append(tmp)
    
    return S

## Euclidean Algorithm Decoding Method

In [15]:
def Euclidean_Algorithm_Error(x_2t,syndrone_poly,t,n):# Error Overhere
    # as + bt = c
    a = x_2t
    b = syndrone_poly
    r_prev = a
    r_cur  = b
    s_prev = [1]
    s_cur  = [0]
    t_prev = [0]
    t_cur  = [1]
    itr = 0
    while poly_deg(r_cur) >= t :
        
        itr = itr +1
        q, rem = Poly_Divide_GF(r_prev,r_cur,n)
        
        r_cur, r_prev = rem, r_cur
        s_cur,s_prev  = Poly_Add_GF( s_prev ,Poly_Mult_GF(q,s_cur,n)) , s_cur
        t_cur,t_prev  = Poly_Add_GF( t_prev ,Poly_Mult_GF(q,t_cur,n)) , t_cur
        
    # x_2t * s_cur + syndrome_poly * t_cur = r_cur
    # t_cur : Error_Locator_Polynomial
    # r_cur : Key_Equation
    return s_cur , t_cur, r_cur

### The Euclidean Result May be different
So we have to normalized the Error Locator to the Normalized Form
$1+\Lambda_1x+\Lambda_2x^2 + ...$

In [16]:
def Euclidean_Result_Normalizer(t_prev,r_prev,n):
    normalized_factor = GF_inv(t_prev[0],n)
    Key_equation       = Poly_Mult_GF([normalized_factor],r_prev,n)
    Error_locator_poly = Poly_Mult_GF([normalized_factor],t_prev,n)
    return Key_equation,Error_locator_poly

## Chien Search

return the error locator reciprocal

In [17]:
def Chien_Search(Error_locator_poly,n):
    exp_form = [GF_log_a_Table_int[val] for val in Error_locator_poly]
    Error_Loc =[]
    for i in range(2**n-1):
        tmp = 0
        for idx,val in enumerate(exp_form):
            if val == None:
                continue
            exp_val = ((idx*i)+val) % (2**n-1)
            tmp = tmp ^ GF_a_exp_Table_int[exp_val]
        if tmp == 0 :
            Error_Loc.append(GF_a_exp_Table_int[i])
                
    return Error_Loc

## Forney's Algorithm Required Function

#### Required  Function
- Polynomial Evaluation
- Differentiation 

### Polynomial Evaluation

In [18]:
def Evaluation(Poly,sub_val,n):
    exp_form = [GF_log_a_Table_int[val] for val in Poly]
    tmp = 0
    exp_sub_val = GF_log_a_Table_int[sub_val]
    for idx,val in enumerate(exp_form):
        if val == None:
            continue
        exp_val = (idx*exp_sub_val+val) % (2**n-1)
        tmp = tmp ^ GF_a_exp_Table_int[exp_val]
    return tmp

In [30]:
n=8
GF_px = np.asarray([1,0,1,1,1,0,0,0,1])
#                   0 1 2 3 4 5 6 7 8
alpha = np.asarray([0,1,0,0,0,0,0,0])
#                   0 1 2 3 4 5 6 7
irreducible_int_form = array2int(GF_px)
alpha_int_form = array2int(alpha)

GF_a_exp_Table_int = GF_table_binary_approach(alpha_int_form,irreducible_int_form,n)
GF_log_a_Table_int = GF_log_a_table_gen(GF_a_exp_Table_int)

Poly_GF_2_8 = [1,0,1,1,1,0,0,0,1]


In [31]:
Evaluation(Poly=Poly_GF_2_8,sub_val=alpha_int_form,n=n)

0

In [33]:
all_root = Chien_Search(Poly_GF_2_8,n)

In [34]:
Poly_exp_view(poly=all_root)

[1, 2, 4, 8, 16, 32, 64, 128]
